In [1]:
!pip install datasets==3.1.0
#load_dataset sometimes hangs on a higher version
!pip install transformers torch evaluate tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


# Preprocessing

In [2]:
from datasets import load_dataset

import torch
import numpy as np
import random

# we set up some seeds so that we can reproduce results
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


# Step 1: Load and parse data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Change train.json / dev.json to the appropriate filepaths =====
def load_data():
    data_files = {"train": "/content/drive/My Drive/Colab Notebooks/all_train.json", "dev": "/content/drive/My Drive/Colab Notebooks/all_dev.json"}
    dataset = load_dataset('json', data_files=data_files)
    train = dataset["train"]
    validation = dataset["dev"]
    return train, validation

In [5]:
class QADataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = 512

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        question = data["questions"][0]["input_text"]
        context = data["contexts"]
        answer = data["answers"][0]
        ans_start = answer['span_start']
        ans_end = answer['span_end']
        type_mapping = {"short": 1, "no_answer": 0}
        ans_type = type_mapping.get(answer.get("input_text", "no_answer"), 0)

        # encode data
        encoded_data = self.tokenizer(question,
                                 context,
                                 add_special_tokens=True,
                                 max_length=self.max_len,
                                 padding='max_length',
                                 truncation=True,
                                 return_attention_mask=True,
                                 return_offsets_mapping=True,
                                 return_tensors="pt")

        offsets = encoded_data.offset_mapping[0].tolist()
        start_token = 0
        end_token = 0
        if ans_type != 0 and ans_start != -1 and ans_end != -1:
            char_start = ans_start
            char_end = ans_end
            start_token = -1
            end_token = -1
            for i, (start, end) in enumerate(offsets):
                if start == 0 and end == 0:
                    continue
                if start_token == -1 and start <= char_start < end:
                    start_token = i
                if end_token == -1 and start <= char_end <= end:
                    end_token = i
                # if both tokens are found no need to continue searching
                if start_token != -1 and end_token != -1:
                    break
            if start_token == -1 or end_token == -1:
                start_token = 0
                end_token = 0
        # print("start token:", start_token)
        # print("end token:", end_token)

        return {
            'input_ids': encoded_data["input_ids"].squeeze(),
            'attention_mask': encoded_data["attention_mask"].squeeze(),
            'start_positions': torch.tensor(start_token, dtype=torch.long),
            'end_positions': torch.tensor(end_token, dtype=torch.long),
            'ans_type': torch.tensor(ans_type, dtype=torch.long),
            'offset_mapping': encoded_data["offset_mapping"].squeeze(),
            'context': context
        }

In [6]:
from torch.utils.data import DataLoader

def preprocess_and_tokenize(data, tokenizer):
    dataset = QADataset(data, tokenizer)
    batch_size = 32
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Step 2: Load model

In [7]:
"""
Some options for BERT model that can be run in colab:

"distilbert-base-uncased",
"distilbert-base-uncased-distilled-squad",
"distilbert-base-cased",
"distilbert-base-cased-distilled-squad",

"""

'\nSome options for BERT model that can be run in colab:\n\n"distilbert-base-uncased",\n"distilbert-base-uncased-distilled-squad",\n"distilbert-base-cased",\n"distilbert-base-cased-distilled-squad",\n\n'

In [8]:
from transformers import DistilBertModel, DistilBertConfig, DistilBertPreTrainedModel
import torch.nn as nn

class DistilBertForQA(nn.Module):
    def __init__(self):
        super(DistilBertForQA, self).__init__()
        self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.start_outputs = torch.nn.Linear(self.model.config.hidden_size, 1)
        self.end_outputs = torch.nn.Linear(self.model.config.hidden_size, 1)
        self.type_outputs = torch.nn.Linear(self.model.config.hidden_size, 5)

    def forward(self, input_ids, token_types=None, attention_mask=None, start_positions=None, end_positions=None, ans_types=None):
        outputs = self.model(input_ids, attention_mask=attention_mask)

        # Extract logits
        start_logits = self.start_outputs(outputs.last_hidden_state).squeeze(-1)
        end_logits = self.end_outputs(outputs.last_hidden_state).squeeze(-1)
        type_logits = self.type_outputs(outputs.last_hidden_state[:, 0, :])

        return start_logits, end_logits, type_logits

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def load_model():
    model = DistilBertForQA()
    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
    return model, tokenizer

In [10]:
def get_loss(start_logits, end_logits, type_logits, start_positions, end_positions, ans_types):
    start_loss = nn.CrossEntropyLoss()(start_logits, start_positions)
    end_loss = nn.CrossEntropyLoss()(end_logits, end_positions)
    type_loss = nn.CrossEntropyLoss()(type_logits, ans_types)
    loss = start_loss + end_loss + type_loss
    return loss

# Step 3 Train

In [12]:
from tqdm.auto import tqdm
def train_loop(model, train_dataloader, validation_dataloader, optimizer, lr_scheduler, num_epochs, device):
    model.to(device)
    train_loss = []

    for epoch in range(num_epochs):
        model.train()
        curr_train_loss = 0
        progress_bar = tqdm(range(len(train_dataloader)))
        for batch in train_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            start_positions = batch["start_positions"].to(device)
            # print("start:", start_positions)
            end_positions = batch["end_positions"].to(device)
            # print("end:", start_positions)
            ans_types = batch['ans_type'].to(device)

            # Calculate loss
            start_logits, end_logits, type_logits = model(
                      input_ids=input_ids,
                      attention_mask=attention_mask,
                      start_positions=start_positions,
                      end_positions=end_positions,
                      ans_types=ans_types)
            loss = get_loss(
                start_logits,
                end_logits,
                type_logits,
                start_positions,
                end_positions,
                ans_types)

            curr_train_loss += loss.item()
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()

            progress_bar.update(1)

        train_loss.append(curr_train_loss / len(train_dataloader))
        print(f"Epoch {epoch+1} training loss: {train_loss[-1]}")

    return train_loss

# Step 4 Eval

In [32]:
from collections import Counter

def eval_loop(model, validation_dataloader, device):
    model.eval()
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    span_preds = []
    span_trues = []
    num_batch = len(validation_dataloader)
    with torch.no_grad():
        for batch in validation_dataloader:
            inputs = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_logits, end_logits, type_logits = model(inputs, attention_mask=attention_mask)

            span_start_trues = batch['start_positions'].to(device)
            span_end_trues = batch['end_positions'].to(device)
            ans_type_trues = batch['ans_type'].to(device)

            batch_size = start_logits.size(0)
            curr_precision = 0
            curr_recall = 0
            curr_f1 = 0

            for i in range(batch_size):
                start_pred = torch.argmax(start_logits[i])
                end_pred = torch.argmax(end_logits[i])
                type_pred = torch.argmax(type_logits[i])
                start_true = span_start_trues[i].item()
                end_true = span_end_trues[i].item()
                type_true = ans_type_trues[i].item()

                index_pred = inputs[i][start_pred:end_pred + 1]
                index_true = inputs[i][start_true:end_true + 1]
                counter_pred = Counter(tokenizer.convert_ids_to_tokens(index_pred))
                counter_true = Counter(tokenizer.convert_ids_to_tokens(index_true))
                true_positive = sum((counter_pred & counter_true).values())
                false_positive = sum((counter_pred - counter_true).values())
                false_negative = sum((counter_true - counter_pred).values())

                if true_positive + false_positive != 0:
                    curr_precision += true_positive / (true_positive + false_positive)
                if true_positive + false_negative != 0:
                    curr_recall += true_positive / (true_positive + false_negative)
                if curr_precision + curr_recall != 0:
                    curr_f1 += 2 * (curr_precision * curr_recall) / (curr_precision + curr_recall)

            total_precision += curr_precision / batch_size
            total_recall += curr_recall / batch_size
            total_f1 += curr_f1 / batch_size

    return total_precision / num_batch, total_recall / num_batch, total_f1 / num_batch

# Run

In [34]:
from transformers import get_scheduler
from torch.optim import AdamW

def main():

  '''Here's the basic structure of the main block -- feel free to add or
  remove parameters/helper functions as you see fit, but all steps here are
  needed and we expect to see precision, recall, and f1 scores printed out'''
  global model, tokenizer, validation_dataloader, device
  device = "cuda" if torch.cuda.is_available() else "cpu"
  batch_size = 32
  num_epochs = 2

  model, tokenizer = load_model()
  model.to(device)
  train, validation = load_data()

  train_dataloader = preprocess_and_tokenize(train, tokenizer)
  validation_dataloader = preprocess_and_tokenize(validation, tokenizer)

  optimizer = AdamW(model.parameters(), lr=5e-5)
  lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=50,
    num_training_steps=len(train_dataloader) * num_epochs
  )

  train_loss = train_loop(
      model,
      train_dataloader,
      validation_dataloader,
      optimizer,
      lr_scheduler,
      num_epochs,
      device)
  print("Overall TRAIN LOSS: ", train_loss)

  precision, recall, f1_score  = eval_loop(
            model=model,
            validation_dataloader=validation_dataloader,
            device=device)

  print("PRECISION: ", precision)
  print("RECALL: ", recall)
  print("F1-SCORE: ", f1_score)

if __name__ == "__main__":
  main()

  0%|          | 0/871 [00:00<?, ?it/s]

Epoch 1 training loss: 4.122598172602779


  0%|          | 0/871 [00:00<?, ?it/s]

Epoch 2 training loss: 2.3576733873030076
Overall TRAIN LOSS:  [4.122598172602779, 2.3576733873030076]
PRECISION:  0.6798551828243032
RECALL:  0.7082178310981581
F1-SCORE:  11.32230496113687


In [ ]:
from transformers import get_scheduler
from torch.optim import AdamW

def log():

  '''Here's the basic structure of the main block -- feel free to add or
  remove parameters/helper functions as you see fit, but all steps here are
  needed and we expect to see precision, recall, and f1 scores printed out'''

  global model, tokenizer, validation_dataloader, device
  precision, recall, f1_score  = eval_loop(
            model=model,
            validation_dataloader=validation_dataloader,
            device=device)

  print("PRECISION: ", precision)
  print("RECALL: ", recall)
  print("F1-SCORE: ", f1_score)

log()